In [1]:
import json
import pandas as pd
import numpy as np
import mesa_reader as mr

In [2]:
lodders03 = pd.read_fwf('mesa_apokasc/chem/lodders03.data', skiprows=5, header=None)
lodders03 = lodders03.rename(columns=dict(enumerate(['Z', 'El', 'A', '%', 'N'])))
lodders03['ElA'] = ''
for i, row in lodders03.iterrows():
    lodders03.at[i, 'ElA'] = row['El'].lower() + str(row['A'])
lodders03 = lodders03.set_index('ElA')
lodders03.info()

<class 'pandas.core.frame.DataFrame'>
Index: 286 entries, h1 to u38
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Z       286 non-null    int64  
 1   El      286 non-null    object 
 2   A       286 non-null    int64  
 3   %       286 non-null    float64
 4   N       286 non-null    object 
dtypes: float64(1), int64(2), object(2)
memory usage: 13.4+ KB


In [3]:
for El in set(lodders03['El']):
    sum_ = sum(lodders03['%'][lodders03['El'] == El])
    if abs(sum_-100) > 1e-6: print(El, f"{sum_-100:.6e}")

O 4.000000e-05
Hg -5.600000e-04
Kr 2.000000e-03
Er 6.000000e-03
Pb 2.000000e+00
Os -3.000000e-04
U -5.500000e-03
Dy 2.000000e-03
Pt 4.000000e-06
K 2.000000e-06
Xe 1.000000e-03
Nd 1.000000e-02
Ru 1.000000e-04


In [4]:
explore_rp_153 = False  # 8/14/2024

if explore_rp_153:
    h = mr.MesaData('prototype_varmix/history_rp_153.data')
    ElAs = [name.removeprefix('surface_') for name in h.bulk_names
        if name.startswith('surface_')]  # and h.data(name)[0] > 0.0]
    for ElA in ElAs:
        print(ElA, h.data(f'surface_{ElA}')[0])

In [5]:
if explore_rp_153:
    # Note: These are mass ratios versus number ratios...

    print(h.data(f'surface_n14')[0] / h.data(f'surface_n15')[0],
          lodders03['%'].loc['n14'] / lodders03['%'].loc['n15'])
    # 253.92082003565034 272.000273000273

    print(h.data(f'surface_ne20')[0] / h.data(f'surface_ne21')[0],
          lodders03['%'].loc['ne20'] / lodders03['%'].loc['ne21'])
    # 397.260834963878 417.1593357271095

In [6]:
# from mesa_apokasc/chem/solmix.ipynb

# with open('mesa_apokasc/chem/isotopes.json', 'r') as f:
#     isotopes = json.load(f)

with open('mesa_apokasc/chem/weights.json', 'r') as f:
    weights = json.load(f)

solmix = pd.read_excel('mesa_apokasc/chem/solmix.xlsx', index_col=0)
elements = list(solmix.index[5:])

# from mesa_apokasc/chem/varmix.ipynb

pp_and_cno_extras = ['h1', 'h2', 'he3', 'he4', 'li7', 'be7', 'b8',
                     'c12', 'c13', 'n13', 'n14', 'n15', 'o14', 'o15', 'o16', 'o17', 'o18',
                     'f17', 'f18', 'f19', 'ne18', 'ne19', 'ne20', 'mg22', 'mg24']

In [7]:
def get_expected_frac(El: str):
    total = 0.0
    for metal in elements[2:]:
        fraction = 10.0 ** solmix.at[metal, 'G98D'] * weights[metal]
        total += fraction
        if metal == El: my_fraction = fraction
    return (my_fraction / total) * 0.015625

In [8]:
if False:  # MESA results based on pp_and_cno_extras.net
    h = mr.MesaData('prototype_varmix/LOGS_start/history.data')
    print(get_expected_frac('F') / h.data(f'surface_f19')[0])  # 0.998226930158158
    print(get_expected_frac('C') / (h.data(f'surface_c12')[0] + h.data(f'surface_c13')[0]))  # 0.9991515528768977
    print(get_expected_frac('N') / (h.data(f'surface_n14')[0] + h.data(f'surface_n15')[0]))  # 0.9984872785405259

In [9]:
# make varmix.net based on lodders03.data

for ElA in lodders03.index:
    El = ElA.rstrip('0123456789').capitalize()
    if El in elements[:30]:  # up to Zn
        if ElA not in pp_and_cno_extras:
            print(ElA, end=', ')

li6, be9, b10, b11, ne21, ne22, na23, mg25, mg26, al27, si28, si29, si30, p31, s32, s33, s34, s36, cl35, cl37, ar36, ar38, ar40, k39, k40, k41, ca40, ca42, ca43, ca44, ca46, ca48, sc45, ti46, ti47, ti48, ti49, ti50, v50, v51, cr50, cr52, cr53, cr54, mn55, fe54, fe56, fe57, fe58, co59, ni58, ni60, ni61, ni62, ni64, cu63, cu65, zn64, zn66, zn67, zn68, zn70, 

In [10]:
make_GS98_zfracs = False  # 8/14/2024
# MESA results based on varmix.net and zfracs = 3

if make_GS98_zfracs:
    hv3 = mr.MesaData('prototype_varmix/history_varmix_zfracs=3.data')

In [11]:
if make_GS98_zfracs:
    z_fraction = dict.fromkeys(elements[2:30], 0.0)

    for ElA in lodders03.index:
        El = ElA.rstrip('0123456789').capitalize()
        if El in elements[2:30]:  # up to Zn
            # print(ElA, h.data(f'surface_{ElA}')[0])
            z_fraction[El] += hv3.data(f'surface_{ElA}')[0] * 64

In [12]:
if make_GS98_zfracs:
    with open('GS98_zfracs.json', 'w') as f:
        json.dump(z_fraction, f)

In [13]:
if not make_GS98_zfracs:  # 8/26/2024
    with open('mesa_apokasc/chem/GS98_zfracs.json', 'r') as f:
        z_fraction = json.load(f)

In [14]:
# make inlist_start with zfracs = 0

for El, frac in z_fraction.items():
    print(f'      z_fraction_{El.lower():2} = {frac:.16e}'.replace('e-0', 'd-'))

      z_fraction_li = 6.2130551359173677d-7
      z_fraction_be = 1.0281235548368182d-8
      z_fraction_b  = 2.9442539432281245d-7
      z_fraction_c  = 1.7234416803930760d-1
      z_fraction_n  = 5.0517137046680288d-2
      z_fraction_o  = 4.6902638225747662d-1
      z_fraction_f  = 2.4884702142252405d-5
      z_fraction_ne = 1.0425122119375173d-1
      z_fraction_na = 2.0832911008088566d-3
      z_fraction_mg = 3.9550850202704046d-2
      z_fraction_al = 3.6164466074365925d-3
      z_fraction_si = 4.4057043329104026d-2
      z_fraction_p  = 4.8776349917941373d-4
      z_fraction_s  = 2.1977896032177386d-2
      z_fraction_cl = 2.8899925908400172d-4
      z_fraction_ar = 3.9185567682208950d-3
      z_fraction_k  = 2.2796890393858311d-4
      z_fraction_ca = 3.8803328813059421d-3
      z_fraction_sc = 2.4547211099022999d-6
      z_fraction_ti = 1.8112790168843204d-4
      z_fraction_v  = 2.3137053052812830d-5
      z_fraction_cr = 1.1033709217569744d-3
      z_fraction_mn = 8.07399633

In [15]:
if make_GS98_zfracs:
    hv0 = mr.MesaData('prototype_varmix/history_varmix_zfracs=0.data')

    for ElA in lodders03.index:
        El = ElA.rstrip('0123456789').capitalize()
        if El in elements[:30]:  # up to Zn
            ratio = hv3.data(f'surface_{ElA}')[0] / hv0.data(f'surface_{ElA}')[0]
            if abs(ratio-1) > 1e-6: print(ElA, ratio)

In [16]:
if make_GS98_zfracs:
    he0 = mr.MesaData('prototype_varmix/history_extras_zfracs=0.data')
    he3 = mr.MesaData('prototype_varmix/history_extras_zfracs=3.data')

    for ElA in pp_and_cno_extras:
        if ElA in ['b8', 'n13', 'o14', 'o15', 'f17', 'f18', 'ne18', 'ne19', 'mg22']: continue
        ratio = he3.data(f'surface_{ElA}')[0] / he0.data(f'surface_{ElA}')[0]
        if abs(ratio-1) > 1e-6: print(ElA, ratio, he3.data(f'surface_{ElA}')[0], he0.data(f'surface_{ElA}')[0])

In [17]:
# from kap/aesopus_maker.py

alpha_elements = ['O', 'Ne', 'Mg', 'Si', 'S', 'Ar', 'Ca']

def CFe_NFe_calc(FeH):
    CFe = 0.268 * FeH**2 + 0.0258 * FeH - 0.00983
    NFe = 0.373 * FeH**2 + 0.373  * FeH + 0.0260
    return CFe, NFe

In [18]:
def vary_z_fraction(AFe: float = None, CNFe: float = None):
    z_fraction_ = z_fraction.copy()

    if AFe is not None:
        for elem in ['O', 'Ne', 'Mg', 'Si', 'S', 'Ar', 'Ca']:
            z_fraction_[elem] *= 10 ** AFe

    if CNFe is not None:
        CFe, NFe = CFe_NFe_calc(CNFe)
        z_fraction_['C'] *= 10 ** CFe
        z_fraction_['N'] *= 10 ** NFe

    # normalization
    total = sum(z_fraction_.values())
    for El in z_fraction_:
        z_fraction_[El] /= total

    for El, frac in z_fraction_.items():
        print(f'      z_fraction_{El.lower():2} = {frac:.16e}'.replace('e-0', 'd-'))
        # print(f'      z_fraction_{El.lower():2} = {z_fraction[El]:.16e}'.replace('e-0', 'd-'))

In [19]:
# vary_z_fraction(AFe=0.1)

CNFe = +0.45  # -0.48, -0.32, -0.16, +0.00, +0.15, +0.30, +0.45
# vary_z_fraction(CNFe=CNFe)

In [20]:
# from chem/varmix.ipynb

df = pd.read_excel('mesa_apokasc/chem/solmix.xlsx', index_col=0)[['G98D']]
df = df.rename(columns={'G98D': 'GS98'})
# elements = list(df.index.values[5:])
GS98 = dict(df['GS98']); del df

In [21]:
# calculate Z/X for varying C and N

def calc_Z_over_X(stdmix: dict, CNFe: float = None) -> float:
    X = 10.0 ** stdmix['H' ] * weights['H' ]
    Y = 10.0 ** stdmix['He'] * weights['He']
    if CNFe is not None:
        CFe, NFe = CFe_NFe_calc(CNFe)

    Z = 0.0
    for metal in elements[2:]:
        fraction = 10.0 ** stdmix[metal] * weights[metal]
        if CNFe is not None:
            if metal == 'C':   fraction *= 10 ** CFe
            elif metal == 'N': fraction *= 10 ** NFe
        Z += fraction

    return Z/X

In [22]:
def calc_Z_over_X_(FeH: float = None) -> float:
    X = 1007844521769.9199
    # Y = 85781480907.22614
    Z_C = 3977254034.668961
    Z_N = 1165028735.404283
    Z_other = 17954706441.722904

    if FeH is not None:
        Z_C *= 10 ** (0.268 * FeH**2 + 0.0258 * FeH - 0.00983)
        Z_N *= 10 ** (0.373 * FeH**2 + 0.373  * FeH + 0.0260 )
    return (Z_C + Z_N + Z_other) / X

for CNFe in [None, -0.48, -0.32, -0.16, +0.00, +0.15, +0.30, +0.45]:
    print(calc_Z_over_X(GS98, CNFe=CNFe) - calc_Z_over_X_(CNFe))

-6.938893903907228e-18
0.0
-6.938893903907228e-18
-3.469446951953614e-18
-3.469446951953614e-18
-3.469446951953614e-18
0.0
-6.938893903907228e-18


In [23]:
# for var_chem_r24cn.py

def Z_OVER_X_OPT(FeH: float) -> float:
    X = 1007844521769.9199
    # Y = 85781480907.22614
    Z_C = 3977254034.668961
    Z_N = 1165028735.404283
    Z_other = 17954706441.722904

    if FeH is not None:
        Z_C *= 10 ** (0.268 * FeH**2 + 0.0258 * FeH - 0.00983)
        Z_N *= 10 ** (0.373 * FeH**2 + 0.373  * FeH + 0.0260 )
    return 0.022903289366476506 / 0.02291721462278181 * (Z_C + Z_N + Z_other) / X

In [24]:
Z_OVER_X_OPT_ = {  # from chem/varmix.xlsx
    -0.48: 0.0231137560860558,  # CN-0.48
    -0.32: 0.0228513802114704,  # CN-0.32
    -0.16: 0.0227769221396675,  # CN-0.16
     0.00: 0.0228862985189309,  # CN+0.00
     0.15: 0.0231710663378548,  # CN+0.15
     0.30: 0.0236677959294306,  # CN+0.30
     0.45: 0.0244394709990096}  # CN+0.45

for FeH in [-0.48, -0.32, -0.16, +0.00, +0.15, +0.30, +0.45]:
    print(Z_OVER_X_OPT(FeH) - Z_OVER_X_OPT_[FeH])

-6.938893903907228e-18
-1.3877787807814457e-17
-2.42861286636753e-17
-3.469446951953614e-18
2.42861286636753e-17
-3.8163916471489756e-17
-3.8163916471489756e-17
